In [ ]:
import os
import pandas as pd
import numpy as np
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import torch
import json
import torch.nn as nn
import matplotlib.pyplot as plt
from rich import print as rprint

from trading.src.user_cache.user_cache import UserCache as user_cache
user = user_cache.load()

In [ ]:

# ---- PARAMETERS ----
ALPACA_API_KEY = user.alpaca_api_key.get_secret_value()
ALPACA_SECRET_KEY = user.alpaca_api_secret.get_secret_value()
TICKERS = ["AAPL"]  # or use ["AAPL", "MSFT", ...] for all tickers
START_DATE = "2000-01-01"
END_DATE = "2023-01-01"
CACHE_FILE = "alpaca_cache.parquet"
EPOCHS = 100
BATCH_SIZE = 16
OVERFITTING_PARAM = 0.05  # e.g., weight decay
SAVE_PARAMS = False
PARAMS_FILE = "run_params.json"
LEARNING_RATE = 2e-5
HOLD_THRESHOLD=0.01

In [ ]:

# ---- DATA COLLECTION & CACHING ----
def fetch_and_cache_data(tickers, start, end, cache_file):
    if os.path.exists(cache_file):
        rprint("Loading data from cache...")
        df = pd.read_parquet(cache_file)
    else:
        rprint("Fetching data from Alpaca...")
        client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET_KEY)
        all_data = []
        for ticker in tickers:
            request_params = StockBarsRequest(
                symbol_or_symbols=ticker,
                timeframe=TimeFrame.Day,
                start=pd.to_datetime(start),
                end=pd.to_datetime(end)
            )
            bars = client.get_stock_bars(request_params).df
            bars['ticker'] = ticker
            all_data.append(bars)
        df = pd.concat(all_data)
        df.to_parquet(cache_file)
    return df

df = fetch_and_cache_data(TICKERS, START_DATE, END_DATE, CACHE_FILE)
rprint(df.head())


In [ ]:

# ---- FEATURE ENGINEERING ----
def add_features(df):
    df = df.copy()
    df['return'] = df['close'].pct_change()
    df['ma5'] = df['close'].rolling(window=5).mean()
    df['ma10'] = df['close'].rolling(window=10).mean()
    df['volatility'] = df['close'].rolling(window=5).std()
    df['target'] = (df['close'].shift(-1) > df['close']).astype(int)  # 1 if next day up, else 0
    df = df.dropna()
    return df

# 0 = SELL, 1 = HOLD, 2 = BUY
def add_trading_signals(df, hold_threshold=0.002):
    df = df.copy()
    df['future_return'] = df['close'].shift(-1) / df['close'] - 1
    # BUY if next day's return > hold_threshold, SELL if < -hold_threshold, else HOLD
    df['signal'] = np.where(df['future_return'] > hold_threshold, 2, 
                    np.where(df['future_return'] < -hold_threshold, 0, 1))
    df = df.dropna()
    return df


In [ ]:

df = add_trading_signals(df, hold_threshold=HOLD_THRESHOLD)
df = add_features(df)

In [ ]:

# ---- DATA VISUALIZATION ----
df_plot = df.reset_index()
plt.figure(figsize=(12,6))
plt.plot(df_plot['timestamp'], df_plot['close'], label='Close Price')
plt.plot(df_plot['timestamp'], df_plot['ma5'], label='MA5')
plt.plot(df_plot['timestamp'], df_plot['ma10'], label='MA10')
plt.title(f"{TICKERS[0]} Price and Moving Averages")
plt.legend()
plt.show()


In [ ]:

# ---- DATA PREPARATION ----
features = ['open', 'high', 'low', 'close', 'volume', 'ma5', 'ma10', 'volatility']
X = df[features].values
y = df['signal'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, shuffle=False
)


In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )
    def forward(self, x):
        return self.net(x)

In [ ]:
class SimpleMLP3(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 3)
        )
    def forward(self, x):
        return self.net(x)

In [ ]:
class ProfitLoss(torch.nn.Module):
    def __init__(self, close_prices):
        super().__init__()
        self.close_prices = torch.tensor(close_prices, dtype=torch.float32).to(device)

    def forward(self, outputs, targets, indices):
        # outputs: [batch, 3], targets: [batch], indices: [batch]
        probs = torch.softmax(outputs, dim=1)  # [batch, 3]

        batch_close = self.close_prices[indices]
        batch_next_close = self.close_prices[indices + 1]
        change = (batch_next_close - batch_close) / batch_close  # percent change

        # profit = [SELL, HOLD, BUY]
        # SELL = price drops → profit
        # BUY = price rises → profit
        # HOLD = 0
        profit_vector = torch.stack([
            -change,                     # SELL
            torch.zeros_like(change),   # HOLD
            change                      # BUY
        ], dim=1)  # [batch, 3]

        expected_profit = torch.sum(probs * profit_vector, dim=1)  # [batch]
        loss = -expected_profit.mean()  # negative to maximize expected profit
        return loss

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = SimpleMLP(X_train.shape[1]).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=OVERFITTING_PARAM)
loss_fn = nn.CrossEntropyLoss()

model = SimpleMLP3(X_train.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=OVERFITTING_PARAM)
profit_loss_fn = ProfitLoss(df['close'].values)


In [ ]:

# ---- TRAINING LOOP ----
def train_model(model, X_train, y_train, epochs, batch_size):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        permutation = np.random.permutation(len(X_train))
        for i in range(0, len(X_train), batch_size):
            indices = permutation[i:i+batch_size]
            batch_x = torch.tensor(X_train[indices], dtype=torch.float32).to(device)
            batch_y = torch.tensor(y_train[indices], dtype=torch.long).to(device)
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = loss_fn(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / (len(X_train) // batch_size)
        rprint(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

# profit-based loss
def train_model_profit(model, X_train, y_train, epochs, batch_size):
    model.train()
    n = len(X_train)
    for epoch in range(epochs):
        total_loss = 0
        batch_count = 0
        permutation = np.random.permutation(n-1)  # n-1 to avoid index error with next day's price
        for i in range(0, n-batch_size, batch_size):  # ensure we have enough samples for next day's price
            batch_indices = permutation[i:i+batch_size]
            batch_x = torch.tensor(X_train[batch_indices], dtype=torch.float32).to(device)
            batch_y = torch.tensor(y_train[batch_indices], dtype=torch.long).to(device)
            batch_indices_tensor = torch.as_tensor(batch_indices, dtype=torch.long, device=device)
            
            optimizer.zero_grad()
            outputs = model(batch_x)
            # Use the custom profit-based loss function
            loss = profit_loss_fn(outputs, batch_y, batch_indices_tensor)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            batch_count += 1
            
        avg_loss = total_loss / batch_count
        rprint(f"Epoch {epoch+1}/{epochs}, Profit-based Loss: {avg_loss:.6f}")

train_model_profit(model, X_train, y_train, EPOCHS, BATCH_SIZE)

# Uncomment to use standard cross-entropy training instead
# train_model(model, X_train, y_train, EPOCHS, BATCH_SIZE)


In [ ]:

# ---- TESTING ----
model.eval()
with torch.no_grad():
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    outputs = model(X_test_tensor)
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    acc = accuracy_score(y_test, preds)
    rprint("Test Accuracy:", acc)
    rprint(classification_report(y_test, preds))


In [ ]:

# ---- PARAMETER SAVING ----
if SAVE_PARAMS:
    params = {
        "tickers": TICKERS,
        "start_date": START_DATE,
        "end_date": END_DATE,
        "features": features,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "overfitting_param": OVERFITTING_PARAM,
        "test_accuracy": acc
    }
    with open(PARAMS_FILE, "w") as f:
        json.dump(params, f, indent=2)
    rprint(f"Parameters saved to {PARAMS_FILE}")

# ---- EXTENSIONS ----
# - To use all tickers, set TICKERS to a list of all tickers.
# - Add more features/signals in add_features().
# - Tune EPOCHS, OVERFITTING_PARAM, etc.
# - Add profit simulation by trading on model predictions.